# Clase: Integración de MySQL con Python 🚀

**🚨 NOTA IMPORTANTE: 🚨**
Te recomendamos encarecidamente tener el **Workbench abierto** mientras ejecutas las celdas de este notebook. Esto te permitirá ver paso a paso cómo se ejecutan las órdenes desde Python en tu base de datos MySQL.

## 1. Configuración Inicial y Conexión a una Base de Datos MySQL desde Python

Antes de sumergirnos en la conexión, es crucial asegurarnos de tener las librerías necesarias instaladas. Si aún no lo has hecho, ejecuta los siguientes comandos en tu terminal (asegúrate de que tu entorno `base` esté activo si usas Anaconda/Jupyter):

```bash
pip install mysql-connector-python
# pip install pandas
```

Ahora, vamos a importar las librerías que utilizaremos a lo largo de esta clase. Siempre que queramos trabajar con librerías, debemos cargarlas previamente:

### 1.1. Importar Librerías Esenciales


In [1]:
import mysql.connector
from mysql.connector import errorcode

### 1.2. Estableciendo la Conexión con `connect()`

El constructor `connect()` es el primer paso fundamental para establecer una comunicación entre tu script de Python y un servidor MySQL. Este constructor devuelve un objeto de tipo `MySQLConnection` que representa nuestra conexión.

Los **argumentos clave** que se utilizan para conectar a una base de datos son:
*   `user`: El nombre de usuario para autenticarse en el servidor MySQL (ej. 'root').
*   `password`: La contraseña correspondiente al usuario.
*   `host`: La dirección IP o el nombre del servidor MySQL al que te conectarás. Por defecto es "localhost" (o '127.0.0.1').
*   `database`: El nombre de la base de datos específica a la que deseas conectarte (ej. 'tienda').
*   `port`: (Opcional) El puerto TCP/IP del servidor MySQL, por defecto es 3306.

Veamos un ejemplo básico de conexión a la base de datos "tienda":

In [2]:
# Hacemos la conexión con el servidor
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1',
    database='tienda'
)

In [4]:
# Imprimimos el objeto de conexión para verificar
print(cnx)

# ¡Importante! Siempre debemos cerrar la conexión cuando ya no la necesitemos
cnx.close()
print("Conexión cerrada.")

Conexión cerrada.


### 1.3. Manejo de Errores en la Conexión con `try-except`

En ocasiones, podemos sufrir errores de conexión (por ejemplo, contraseña incorrecta o base de datos inexistente). Es una buena práctica añadir **manejo de excepciones** usando `try-except` para evitar que nuestro código se detenga abruptamente y para proporcionar mensajes de error útiles. El módulo `errorcode` nos permite identificar tipos específicos de errores.


In [9]:
try:
    # Intenta hacer la conexión con la base de datos 'tienda'
    cnx = mysql.connector.connect(
        user='root',
        password='MySQLPassword2023!',
        host='127.0.0.1',
        database='tienda'
    )
    print("¡Conexión exitosa a MySQL!") #

except mysql.connector.Error as err:
    # Si es un error de acceso denegado (ej. contraseña o usuario incorrecto)
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña.") #
    # Si la base de datos no existe
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe.") #
    # Para cualquier otro tipo de error
    else:
        print(err) #
        print("Código de Error:", err.errno) #
        print("SQLSTATE", err.sqlstate) #
        print("Mensaje", err.msg) #
else:
    # Si la conexión fue exitosa, cierra la conexión
    cnx.close()
    print("Conexión cerrada.")

¡Conexión exitosa a MySQL!
Conexión cerrada.


### 1.4. Realización de Queries con `cursor()` y `execute()`

Una vez que tenemos la conexión, el siguiente paso es ejecutar consultas SQL. Esto se logra mediante el uso del **método `cursor()`** y el **método `execute()`**.
*   **`cursor()`**: Crea un objeto cursor, que es como la "llave" para interactuar entre Python y MySQL y es esencial para ejecutar consultas SQL.
*   **`execute()`**: Envía una consulta SQL al servidor MySQL y la ejecuta. El resultado se almacena en el cursor si todo va correctamente.

Podemos pasar cualquier tipo de query a `execute()`: crear tablas, alterar tablas, insertar datos, etc..

**Ejemplo: Mostrar tablas de una base de datos**

In [10]:
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1',
    database='tienda'
)

mycursor = cnx.cursor() # Iniciamos el cursor

mycursor.execute("SHOW TABLES") # Ejecutamos la query

# El cursor se convierte en un iterable al que podemos acceder, por ejemplo, con un bucle for
print("Tablas en la base de datos 'tienda':")
for x in mycursor:
    print(x) # Devuelve cada resultado como una tupla

cnx.close()

Tablas en la base de datos 'tienda':
('customers',)
('employees',)
('offices',)
('order_details',)
('orders',)
('payments',)
('product_lines',)
('products',)


**Ejemplo: Crear una base de datos**

Al crear una base de datos, no indicamos ninguna base de datos en la conexión inicial.


In [16]:
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1'
)

mycursor = cnx.cursor()

try:
    mycursor.execute("CREATE DATABASE BD_pruebas")
    print("Base de datos 'BD_pruebas' creada exitosamente.")
except mysql.connector.Error as err:
    print(err)
    print("Código de Error:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Mensaje", err.msg)

cnx.close()
# Si ejecutas esta celda dos veces, obtendrás un aviso de que la base de datos ya existe.


1007 (HY000): Can't create database 'BD_pruebas'; database exists
Código de Error: 1007
SQLSTATE HY000
Mensaje Can't create database 'BD_pruebas'; database exists


**Ejemplo: Crear una tabla**


In [17]:
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1',
    database='BD_pruebas' # Ahora nos conectamos a la nueva DB
)

mycursor = cnx.cursor()

try:
    mycursor.execute("CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))")
    print("Tabla 'customers' creada exitosamente en 'BD_pruebas'.")
except mysql.connector.Error as err:
    print(err)
    print("Código de Error:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Mensaje", err.msg)

cnx.close()

Tabla 'customers' creada exitosamente en 'BD_pruebas'.


**Ejemplo: Insertar un único registro - Uso de `commit()` y `rollback()`**

Para las sentencias `INSERT`, `UPDATE`, `DELETE`, es **necesario usar `cnx.commit()`** para que los cambios se efectúen permanentemente en la base de datos. Si no se llama a este método, las modificaciones no se guardarán.

El método `rollback()` permite **deshacer una transacción** si aún no se ha ejecutado `commit()` y los datos introducidos son incorrectos.


In [ ]:
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1',
    database='BD_pruebas'
)

mycursor = cnx.cursor()

query = "INSERT INTO customers (name, address) VALUES (%s, %s)" # Usamos %s como placeholders
val = ("Ana", "Calle 21") # Los valores se pasan como una tupla

try:
    mycursor.execute(query, val) # execute() recibe la query y los valores
    cnx.commit() # Guarda los cambios en la base de datos
    print(mycursor.rowcount, "registro insertado.") # rowcount devuelve el número de filas afectadas
except mysql.connector.Error as err:
    print(err)
    print("Código de Error:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Mensaje", err.msg)

cnx.close()


1 registro insertado.


In [ ]:
# Ejemplo cuando el commit va bien !!!
print("\nDemostración de rollback:")

cnx = mysql.connector.connect(user='root', password='MySQLPassword2023!', host='127.0.0.1', database='BD_pruebas')
mycursor = cnx.cursor()
sql_multi = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val_multi = [
    ('Rocio', 'Apple st 652'),
    ('Sara', 'Mountain 21'),
    ('Pedro', 'Valley 345')
]

try:
    mycursor.executemany(sql_multi, val_multi)
    # Si algo sale mal aquí con alguno de los valores nos vamos al except.
except:
    cnx.rollback()  # Cancela el INSERT
    print("inserción cancelada")
else: # Se ejecuta si todo el try salió bien
    cnx.commit()
    print(mycursor.rowcount, "valores insertados.")
finally: # Se ejecuta siempre
    cnx.close()



Demostración de rollback:
inserción cancelada


In [ ]:
# Ejemplo cuando salta un error en el try a la hora de insertar y nos vamos al rollback !!!
print("\nDemostración de rollback:")

cnx = mysql.connector.connect(user='root', password='MySQLPassword2023!', host='127.0.0.1', database='BD_pruebas')
mycursor = cnx.cursor()
sql_multi = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val_multi = [
    ('Rocio', 'Apple st 652'),
    ('Mountain 21'), # Hemos eliminado uno de los valores para forzar el error
    ('Pedro', 'Valley 345')
]

try:
    mycursor.executemany(sql_multi, val_multi)
    # Si algo sale mal aquí con alguno de los valores nos vamos al except.
except:
    cnx.rollback()  # Cancela el INSERT
    print("inserción cancelada")
else: # Se ejecuta si todo el try salió bien
    cnx.commit()
    print(mycursor.rowcount, "valores insertados.")
finally: # Se ejecuta siempre
    cnx.close()


**Ejemplo: Insertar múltiples registros con `executemany()`**

Si necesitas insertar varias filas, usa `executemany()` con una lista de tuplas que contenga todos los datos.


In [20]:
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1',
    database='BD_pruebas'
)

mycursor = cnx.cursor()

sql_multi = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val_multi = [
    ('Rocio', 'Apple st 652'),
    ('Juana', 'Mountain 21'),
    ('Pedro', 'Valley 345')
] # Lista de tuplas

try:
    mycursor.executemany(sql_multi, val_multi) #
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.") #
except mysql.connector.Error as err:
    print(err)

cnx.close()

3 registro/s insertado/s.


**Ejemplo: Eliminar registros**


In [21]:
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1',
    database='BD_pruebas'
)

mycursor = cnx.cursor()

sql_delete = "DELETE FROM customers WHERE address = 'Calle 21'" 

try:
    mycursor.execute(sql_delete)
    cnx.commit()
    print(mycursor.rowcount, "registro/s eliminado/s.")
except mysql.connector.Error as err:
    print(err)

cnx.close()

1 registro/s eliminado/s.


**Ejemplo: Actualizar registros**


In [29]:
cnx = mysql.connector.connect(
    user='root',
    password='MySQLPassword2023!',
    host='127.0.0.1',
    database='BD_pruebas'
)

mycursor = cnx.cursor()

sql_update = "UPDATE customers SET address = 'Canyon 123' WHERE address = 'Valley 345'" #

try:
    mycursor.execute(sql_update)
    cnx.commit()
    print(mycursor.rowcount, "registro/s modificado/s.") #
except mysql.connector.Error as err:
    print(err)

cnx.close()

1 registro/s modificado/s.


## 2. Acceso a los Resultados de Nuestras Consultas

Una vez que hemos ejecutado nuestras consultas, es momento de aprender cómo acceder a los resultados de manera eficiente para poder manipular y analizar la información.

### 2.1. Acceder a Resultados de Uno en Uno con `fetchone()`

El método `fetchone()` recupera **la primera fila** del resultado de la consulta y **avanza el cursor** al siguiente registro. Esto significa que cada vez que lo uses en el mismo cursor, te mostrará un dato diferente, uno a uno.


In [30]:
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="MySQLPassword2023!",
    database="tienda"
)

mycursor = cnx.cursor()
mycursor.execute("SELECT * FROM employees") # Ejecutamos una query en la tabla 'employees'

# Primera llamada a fetchone()
myresult_1 = mycursor.fetchone()
print("El resultado 1 es: ", myresult_1) #



El resultado 1 es:  (1002, 'Murphy', 'Diane', 'x5800', 'dmurphy@classicmodelcars.com', '1', None, 'President')


In [31]:
# Segunda llamada a fetchone() en el mismo cursor
myresult_2 = mycursor.fetchone()
print("El resultado 2 es: ", myresult_2) #

El resultado 2 es:  (1056, 'Patterson', 'Mary', 'x4611', 'mpatterso@classicmodelcars.com', '1', 1002, 'VP Sales')


In [32]:
mycursor.fetchone()

(1076,
 'Firrelli',
 'Jeff',
 'x9273',
 'jfirrelli@classicmodelcars.com',
 '1',
 1002,
 'VP Marketing')

In [33]:
mycursor.fetchone()


(1088,
 'Patterson',
 'William',
 'x4871',
 'wpatterson@classicmodelcars.com',
 '6',
 1056,
 'Sales Manager (APAC)')

In [34]:
# Ojo: Si no cerramos la conexión, cada vez que ejecutemos fetchone() pasará al siguiente registro.

cnx.close()

### 2.2. Acceder a Todos los Resultados con `fetchall()`

Si deseas obtener **todos los resultados** devueltos por una consulta SQL de una sola vez, puedes utilizar el método `fetchall()`. Este método devuelve los resultados como una **lista de tuplas**, donde cada tupla representa una fila de la consulta.


In [ ]:
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="MySQLPassword2023!",
    database="tienda"
)

mycursor = cnx.cursor()
mycursor.execute("SELECT * FROM employees")

# Obtenemos todos los resultados
myresults_all = mycursor.fetchall() # Devuelve lista de tuplas
print("Todos los resultados (lista de tuplas):")
print(myresults_all) #




Todos los resultados (lista de tuplas):
[(1002, 'Murphy', 'Diane', 'x5800', 'dmurphy@classicmodelcars.com', '1', None, 'President'), (1056, 'Patterson', 'Mary', 'x4611', 'mpatterso@classicmodelcars.com', '1', 1002, 'VP Sales'), (1076, 'Firrelli', 'Jeff', 'x9273', 'jfirrelli@classicmodelcars.com', '1', 1002, 'VP Marketing'), (1088, 'Patterson', 'William', 'x4871', 'wpatterson@classicmodelcars.com', '6', 1056, 'Sales Manager (APAC)'), (1102, 'Bondur', 'Gerard', 'x5408', 'gbondur@classicmodelcars.com', '4', 1056, 'Sale Manager (EMEA)'), (1143, 'Bow', 'Anthony', 'x5428', 'abow@classicmodelcars.com', '1', 1056, 'Sales Manager (NA)'), (1165, 'Jennings', 'Leslie', 'x3291', 'ljennings@classicmodelcars.com', '1', 1143, 'Sales Rep'), (1166, 'Thompson', 'Leslie', 'x4065', 'lthompson@classicmodelcars.com', '1', 1143, 'Sales Rep'), (1188, 'Firrelli', 'Julie', 'x2173', 'jfirrelli@classicmodelcars.com', '2', 1143, 'Sales Rep'), (1216, 'Patterson', 'Steve', 'x4334', 'spatterson@classicmodelcars.com', 

In [40]:
type(myresults_all)

list

In [37]:
# Podemos iterar sobre esta lista de tuplas para acceder a cada registro individualmente
print("\nAccediendo a cada fila individualmente:")
for row in myresults_all:
    print(row) #

cnx.close()


Accediendo a cada fila individualmente:
(1002, 'Murphy', 'Diane', 'x5800', 'dmurphy@classicmodelcars.com', '1', None, 'President')
(1056, 'Patterson', 'Mary', 'x4611', 'mpatterso@classicmodelcars.com', '1', 1002, 'VP Sales')
(1076, 'Firrelli', 'Jeff', 'x9273', 'jfirrelli@classicmodelcars.com', '1', 1002, 'VP Marketing')
(1088, 'Patterson', 'William', 'x4871', 'wpatterson@classicmodelcars.com', '6', 1056, 'Sales Manager (APAC)')
(1102, 'Bondur', 'Gerard', 'x5408', 'gbondur@classicmodelcars.com', '4', 1056, 'Sale Manager (EMEA)')
(1143, 'Bow', 'Anthony', 'x5428', 'abow@classicmodelcars.com', '1', 1056, 'Sales Manager (NA)')
(1165, 'Jennings', 'Leslie', 'x3291', 'ljennings@classicmodelcars.com', '1', 1143, 'Sales Rep')
(1166, 'Thompson', 'Leslie', 'x4065', 'lthompson@classicmodelcars.com', '1', 1143, 'Sales Rep')
(1188, 'Firrelli', 'Julie', 'x2173', 'jfirrelli@classicmodelcars.com', '2', 1143, 'Sales Rep')
(1216, 'Patterson', 'Steve', 'x4334', 'spatterson@classicmodelcars.com', '2', 1143

In [41]:
# primero tenemos que importar la librería

import pandas as pd  # El "as" es como darle un alias a la biblioteca que voy a usar...te recuerda al AS de las query de SQL?

In [44]:
# hacemos la conexión con el servidor
cnx = mysql.connector.connect(user='root', password='MySQLPassword2023!',
                              host='127.0.0.1',
                              database='tienda')

# iniciamos el cursor
mycursor = cnx.cursor()

# ejecutamos nuestra query
mycursor.execute("SELECT * FROM employees")

# le decimos que nos devuelva todos los resultados y los almacenamos en una variable llamada myresult
myresult = mycursor.fetchall()

#Creamos un dataframe con los resultados de la consulta SQL almacenados en myresult. Si os fijáis le estamos pasando un parámetro llamado "columns" donde estamos especificando cuáles son las columnas de lo que será nuestro dataframe
df = pd.DataFrame(myresult, columns = ['ID', 'Nombre', 'Apellido','Email','Telefono','Direccion','Ciudad','Pais'])

#Cerramos la conexion
cnx.close()

In [45]:
df

,ID,Nombre,Apellido,Email,Telefono,Direccion,Ciudad,Pais
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002.0,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056.0,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056.0,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056.0,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143.0,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143.0,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143.0,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep
